In [2]:
# import stanza

# stanza.download('en') # download English model
# nlp = stanza.Pipeline('en') # initialize English neural pipeline
# doc = nlp("Barack Obama was born in Hawaii.")

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [4]:
file_path = "data/accepted_2007_to_2018Q4.csv"
# data = pd.read_csv(file_path)
# data

In [ ]:
from src.data.features import select_features, features, DataType


df = pd.read_csv(file_path)
df = select_features(df)

In [6]:
# df

In [7]:
num_features = [ ft for ft, type in features.items() if type in [DataType.NUMERICAL, DataType.NUMERICAL_LOG] ]
cat_features = [ ft for ft, type in features.items() if type in [DataType.CATEGORICAL] and not ft == "loan_status"]
# num_features.remove("emp_length")
# cat_features.append("emp_length")

In [ ]:
cat_features

In [19]:
df_2 = df.copy()

In [25]:
one_hot = pd.get_dummies(df_2, columns=cat_features, drop_first=True)

In [32]:
one_hot.drop(columns="desc", inplace=True)

In [22]:
df_2 = df_2.join(one_hot)

In [12]:
df_num_paidoff = df[df["loan_status"].apply(lambda x: "Fully Paid" == x)][num_features]
df_num_default = df[df["loan_status"].apply(lambda x: "Charged Off" == x)][num_features]

In [13]:
# df2 = df[df["loan_status"].apply(lambda x: x in ["Fully Paid", "Charged Off"])]
# df2["num_loan_status"] = df["loan_status"].apply(lambda x: 1 if x == "Fully Paid" else 0)

In [29]:
from sklearn.model_selection import train_test_split

def split_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    ftrs = list(df.columns)
    # ftrs = list(features.copy().keys())
    ftrs.remove("loan_status")
    X, y = df[ftrs], df[["loan_status"]]
    X_train, X_devtest, y_train, y_devtest = train_test_split(X, y, test_size=.2)
    X_dev, X_test, y_dev, y_test = train_test_split(X_devtest, y_devtest, test_size=.5)

    return X_train, X_dev, X_test, y_train, y_dev, y_test

In [33]:
X_train, X_dev, X_test, y_train, y_dev, y_test = split_data(one_hot)

In [45]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train, y_train)

In [ ]:
y_train_resampled.mean()

In [ ]:
from scipy import stats

pbc = df[num_features].corrwith(df["loan_status"].astype("float"), method=stats.pointbiserialr)

In [ ]:
pbc.T

In [ ]:
df_num_default.describe().loc[['mean', "50%", 'std']].T

In [ ]:
df_num_paidoff.describe().loc[['mean', "50%", 'std']].T

In [ ]:
len(df_num_default) + len(df_num_paidoff)

In [ ]:
df[num_features].corr()

In [ ]:
df["loan_status"].value_counts(df["loan_status"].unique)